# Caracterização de eletrofáceis por meio de inteligência artificial com abordagem supervisionada
## Reclassificação da litologia metamorfica em xisto ou filito

- **Nome do Arquivo**: tratamento_dados_metamorfica.ipynb
- **Autor**: Rafael Takeguma Goto
- **Versão**: 1.0
- **Data de Criação**: 11/07/2024
- **Data de Modificação**: 12/07/2024

### Descrição: 
Este notebook contém a etapa do trabalho referente à reclassificação da litologia METAMORFICA NAO IDE em XISTO ou FILITO.

Para reclassificar a metamorfica, será utilizado um modelo de classificação de floresta aleatória, que utilizará as propriedades de perfis dos registros de xisto e filito para aprender e ser capaz de rotular a metamorfica como uma dessas duas litologias.

### Leitura do conjunto de dados

O conjunto de dados é constituído por um arquivo csv, que é resultado da concatenação dos arquivos csv gerados a partir da fusão da extração dos arquivos dlis com os arquivos agp de cada poço, seguida da aplicação dos filtros de constantes, nulos e perfis (DCAL e DRHO).

In [1]:
import pandas as pd

path_pocos_pre_transicao = 'Dados-filtrados-concatenados\pocos_pre_transicao.csv'

dados_concatenados_leitura = pd.read_csv(path_pocos_pre_transicao)

dados_concatenados = pd.DataFrame(dados_concatenados_leitura)

### Separa os registros de XISTO e FILITO
Separa as linhas (registros) em que a litologia é XISTO ou FILITO para treinar o modelo com base nesses dados.

In [2]:
dados_xisto_filito = dados_concatenados[(dados_concatenados['Litologia'] == 'XISTO') | (dados_concatenados['Litologia'] == 'FILITO')]

### Divide os dados em treinamento e teste
Divide os dados armazenados em dados_xisto_filito em 2/3 para treinamento do modelo, e 1/3 para teste do modelo.

In [3]:
from sklearn.model_selection import train_test_split

# Define as variáveis que serão utilizadas (propriedades de perfis) 
propriedades = ['GR', 'DCAL', 'RESD', 'DT', 'RHOB', 'DRHO', 'NPHI', 'PE']

# Separa as variáveis (propriedades de perfis) e armazena em X
X = dados_xisto_filito[propriedades]

# Separa a variável alvo (litologia) e armazena em y
y = dados_xisto_filito['Litologia']

# Divide os dados em 2/3 para treinamento e 1/3 para teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=42)

### Treinamento e teste do modelo
Utiliza os dados de treinamento armazenados em X_train e y_train para treinar o modelo de classificação. De forma semelhante, utiliza os dados de teste armazenados em X_test e y_test para testar a acurácia do modelo.

In [4]:
from sklearn.ensemble import RandomForestClassifier

# Treina o modelo
modelo = RandomForestClassifier()
modelo.fit(X_train, y_train)

# Testa o modelo
acuracia = modelo.score(X_test, y_test)
print(f"Acurácia do modelo: {acuracia:2f}")

Acurácia do modelo: 1.000000


### Reclassificação dos registros de metamorfica
Utiliza o modelo treinado acima para prever se os registros que possuem METAMORFICA NAO IDE como litologia tem afinidade para XISTO ou FILITO.

In [5]:
# Separa as linhas (registros) em que a litologia é METAMORFICA NAO IDE 
dados_metamorfica = dados_concatenados[dados_concatenados['Litologia'] == 'METAMORFICA NAO IDE']

# Separa as propriedades das linhas em que a litologia é METAMORFICA NAO IDE 
propriedades_metamorfica = dados_metamorfica[X.columns]

# Realiza a reclassificação da metamorfica para xisto ou filito, com base nas propriedades
predicoes = modelo.predict(propriedades_metamorfica)

# Insere os registros reclassificados em dados_concatenados 
dados_concatenados.loc[dados_concatenados['Litologia'] == 'METAMORFICA NAO IDE', 'Litologia'] = predicoes

### Novos registros de xisto e filito
585 registros de metamorfica foram reclassificados. Segue abaixo a quantidade de registros reclassificados como filito e xisto, respectivamente. 


In [6]:
import numpy as np

# Contagem de ocorrências de cada elemento único
elementos_unicos, contagem = np.unique(predicoes, return_counts=True)

# Exibindo os resultados
for elemento, qtd in zip(elementos_unicos, contagem):
    print(f"{elemento}: {qtd} registro(s)")

FILITO: 5 registro(s)
XISTO: 580 registro(s)


### Boxplots dos registros reclassificados
Boxplots das propriedades dos registros de metamorfica que foram reclassificados em xisto ou filito.

In [7]:
# Cria um dataframe para armazenar as litologias resultantes da reclassificação
litologias_reclassificadas = pd.DataFrame(predicoes, columns=['Litologia'])

# Cria um dataframe com as propriedades das linhas reclassificadas com o index resetado
propriedades_reclassificadas = propriedades_metamorfica.reset_index(drop=True)

# Concatena as litologias reclassificadas com as suas propriedades respectivas
linhas_reclassificadas = pd.concat([litologias_reclassificadas, propriedades_reclassificadas], axis=1)

In [14]:
import matplotlib.pyplot as plt 

def cria_boxplot(df, litologia, propriedade):
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(111)

    # Filtra o dataframe para ficar apenas com os dados da litologia passada como argumento
    dados_litologia = df[(df['Litologia'] == litologia)]

    dados_propriedade = dados_litologia[propriedade]

    # Criar o plot da propriedade
    ax.boxplot(dados_propriedade)
    
    # Adiciona rótulos para os eixos e um título
    ax.set_ylabel(propriedade, fontweight='bold')
    ax.set_title(f'Boxplot da propriedade {propriedade} para metamorfica reclassificada como {litologia}', fontweight='bold')
 
    # Remove traços do eixo x
    ax.tick_params(axis='x', which='both', bottom=False, top=False) 

    ax.set_xticklabels([''])

    # Ajusta layout para que tudo caiba na figura
    plt.tight_layout() 

    return fig    

In [15]:
import os
from matplotlib.backends.backend_pdf import PdfPages

# Litologias envolvidas na reclassificação
litologias = ['XISTO', 'FILITO']

propriedades = [
    'DCAL',
    'GR', 
    'RESD',
    'DT',
    'RHOB',
    'DRHO',
    'NPHI',
    'PE'
]

# Caminho da pasta onde os arquivos PDF serão armazenados
pasta_principal = 'Boxplots-reclassificacao/'
os.makedirs(pasta_principal, exist_ok=True)

# Itera sobre todas as litoligas
for litologia in litologias:
    # Cria pasta para a litologia
    pasta_litologia = f'Boxplots-reclassificacao/{litologia}/'
    os.makedirs(pasta_litologia, exist_ok=True)

    # Itera sobre todas as propriedades
    for propriedade in propriedades:
        # Define o nome do arquivo pdf para a propriedade atual
        pdf_propriedade = os.path.join(pasta_litologia, f'{propriedade}.pdf')

        with PdfPages(pdf_propriedade) as pdf:
            fig = cria_boxplot(linhas_reclassificadas, litologia, propriedade)
                
            # Salva o gráfico no arquivo PDF
            pdf.savefig(fig) 
            plt.close(fig)

### Exporta arquivo csv após reclassificação

In [17]:
dados_concatenados.to_csv('Dados-filtrados-concatenados/pocos_pre_transicao_v2.csv')